<h2> 1. We treat motor UPDRS score as the response and use a variety of regression models to predict
the motor UPDRS score of patients using six biomedical voice measures. </h2>

<table><tr><th>Variable</th> <th>Description</th></tr> 
<tr><td>motor_updrs</td> <td> modified clinician’s motor UPDRS score</td></tr>
<tr><td>Abs,PPQ5</td> <td> Measures of variation in fundamental frequency</td></tr>
<tr><td>dB,APQ11</td> <td> Measures of variation in amplitude</td></tr>
<tr><td>NHR</td> <td>A measure of ratio of noise to tonal components in the voice</td></tr>
<tr><td>RPDE</td> <td>A nonlinear dynamical complexity measure</td></tr>
</table>

Table 1: Description of the response and predictors in park.csv file. The response is in the first
row and the remaining rows describe the biomedical voice measures.

Answer the following questions based on different types of regression models introduced in the
class. Evaluate their predictive performance using mean square prediction error (MSPE) estimates
based on 10-fold cross-validation.

<h3>1. (10 pts) How does a linear regression model perform when predicting motor UPDRS scores using
the given biomedical voice measures? Evaluate the model’s predictive performance</h3>

In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


In [19]:
# Load the dataset
data = pd.read_csv('park.csv')

data.head(5)

,motor_updrs,Abs,PPQ5,dB,APQ11,NHR,RPDE
0,4.234932,0.000017,0.00137,0.136,0.00886,0.011626,0.50668
1,2.014135,0.000033,0.00339,0.225,0.01323,0.061012,0.50328
2,3.808876,0.000044,0.00272,0.287,0.02524,0.017954,0.62443
3,4.147702,0.000063,0.00891,0.758,0.05491,0.089232,0.57429
4,2.270910,0.000105,0.00581,0.404,0.02216,0.031666,0.55557


In [20]:
# Split the dataset into features and target variable
X = data.iloc[:, 1:].values  # Features
# X = data.iloc[1:, 1:].values  # Features

y = data.iloc[:,0].values   # Target variable
#y = data.iloc[0, 1:].values   # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Convert data to float
X_train = X_train.astype(float)
X_test = X_test.astype(float)
y_train = y_train.astype(float)
y_test = y_test.astype(float)

X = X_train
y = y_train

# Function to calculate list of mean squared prediction errors done by k-fold cross validation
# Inputs: k-fold division, the model, X matrix and y matrix 
# Output: List of 9-folds mspe values
# Process: Divide the data into k folds. For each fold, train the data on other k-1 folds
# and calculate mspe on the current fold. Store the values in an array
def kFoldMspeCalc(kFold, model, X, y):
    # Define a matrix for storing mean square pred errors
    mspeErrors = []
    
    # Perform 10-fold cross-validation
    for train_index, test_index in kFold.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Fit the model
        model.fit(X_train, y_train)
        
        # Predict the target variable
        y_pred = model.predict(X_test)
        
        # Cross Validation Errors
        mspeErrors.append(mean_squared_error(y_test,y_pred))
    
    return mspeErrors

def testingMSPE(model, X_testing, y_testing):
    # Fit the model
    model.fit(X_train, y_train)
        
    # Predict the target variable
    y_pred = model.predict(X_testing)

    return(mean_squared_error(y_testing,y_pred))

In [21]:
# Initialize Linear Regression model
model = LinearRegression()

# Define 10-fold cross-validation with shuffling and seed for reproducability
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Get list of mspes
crossValErrors = kFoldMspeCalc(kf, model, X, y)

# Print mean MSPE
print("Linear Regression Model MSPE:", np.mean(crossValErrors))

print("Fitting on Testing Data: ", testingMSPE(model, X_test, y_test))

Linear Regression Model MSPE: 0.9642768220816269
Fitting on Testing Data:  1.1802119869996937


An average MSPE of 1.037 in 10 fold cross training data is achieved. On testing data, the MSPE is 1.18

<h3>2. (20 pts) Extend the previous model to a polynomial regression of degree 2. Extend it further us-
ing the lasso and ridge penalties and principal components regression. Compare their predictive
performance with the previous model.</h3>

In [22]:
from sklearn.preprocessing import PolynomialFeatures

# Initialize Linear Regression model
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X)

crossValErrors = kFoldMspeCalc(kf, model, X_poly, y)

# Print mean MSPE
print("Polynomial Regression Model MSPE:", np.mean(crossValErrors))

print("Fitting on Testing Data: ", testingMSPE(model, X_test, y_test))

Polynomial Regression Model MSPE: 1.4453703001709761
Fitting on Testing Data:  1.1802119869996937


In [23]:
from sklearn.linear_model import Lasso, Ridge;


for alpha in [0.001, 0.002, 0.003, 0.004, 0.005 ]:
    model = Lasso(alpha=alpha)
    crossValErrors = kFoldMspeCalc(kf, model, X, y)
    print("Lasso Regression Model MSPE at alpha = {:}:".format(alpha), np.mean(crossValErrors))

    print("Fitting on Testing Data: ", testingMSPE(model, X_test, y_test))


Lasso Regression Model MSPE at alpha = 0.001: 0.9592875498055763
Fitting on Testing Data:  1.160594171255275
Lasso Regression Model MSPE at alpha = 0.002: 0.958217827402993
Fitting on Testing Data:  1.1624198944398119
Lasso Regression Model MSPE at alpha = 0.003: 0.95860954145864
Fitting on Testing Data:  1.1644268937888511
Lasso Regression Model MSPE at alpha = 0.004: 0.9593607059585854
Fitting on Testing Data:  1.1666160260195362
Lasso Regression Model MSPE at alpha = 0.005: 0.9603093728997983
Fitting on Testing Data:  1.1689872911318682


In [24]:
for alpha in [0.00005, 0.000075, 0.0001, 0.001, 0.0015]:
    model = Ridge(alpha=alpha)

    crossValErrors = kFoldMspeCalc(kf, model, X, y)

    print("Ridge Regression Model MSPE at alpha = {:5f}:".format(alpha), np.mean(crossValErrors))
    print("Fitting on Testing Data: ", testingMSPE(model, X_test, y_test))

Ridge Regression Model MSPE at alpha = 0.000050: 0.9750276771726953
Fitting on Testing Data:  1.1626405044632477
Ridge Regression Model MSPE at alpha = 0.000075: 0.9750590333182718
Fitting on Testing Data:  1.162583761266616
Ridge Regression Model MSPE at alpha = 0.000100: 0.9750756982454458
Fitting on Testing Data:  1.1625401768388757
Ridge Regression Model MSPE at alpha = 0.001000: 0.9750409471383943
Fitting on Testing Data:  1.1616643023940239
Ridge Regression Model MSPE at alpha = 0.001500: 0.9749398913476307
Fitting on Testing Data:  1.161351479339823


In [25]:
from sklearn.decomposition import PCA 
from sklearn.pipeline import Pipeline 


reg = LinearRegression() 


for n_components in [6,5,4,3,2,1]:
    pca = PCA(n_components=n_components)

    pipeline = Pipeline(steps=[('pca', pca), 
                           ('reg', reg)]) 
    crossValErrors = kFoldMspeCalc(kf, pipeline, X, y)

    print("Principal Component Regression Model MSPE with "+str(n_components)+" components:", np.mean(crossValErrors))
    print("Fitting on Testing Data: ", testingMSPE(model, X_test, y_test))

Principal Component Regression Model MSPE with 6 components: 0.9642768220816261
Fitting on Testing Data:  1.161351479339823
Principal Component Regression Model MSPE with 5 components: 0.9751073434933973
Fitting on Testing Data:  1.161351479339823
Principal Component Regression Model MSPE with 4 components: 0.9781503117944551
Fitting on Testing Data:  1.161351479339823
Principal Component Regression Model MSPE with 3 components: 0.9649861842418479
Fitting on Testing Data:  1.161351479339823
Principal Component Regression Model MSPE with 2 components: 0.9583892408779737
Fitting on Testing Data:  1.161351479339823
Principal Component Regression Model MSPE with 1 components: 1.7402786517183522
Fitting on Testing Data:  1.161351479339823


Polynomial features have the worst training error @ 1.44 and ridge, lasso and PCA haveing rate of 0.95. 

However all models have a comparable training error of around 1.16. Lasso regression seem to perform best here

<h3>3. (10 pts) How does using the polynomial kernels of degrees 1, 2, 10, and 20 in kernel regression
affect the prediction accuracy of motor UPDRS scores? Does the polynomial kernel beat the
performance of the previous models for some degree? Justify the similarity between polynomial
regressions used in the previous questions and kernel regressions used in this question.</h3>

In [26]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.kernel_ridge import KernelRidge

# Ridge penalty values
alphas = [0, 0.1, 0.5, 1]

# Polynomial degree values
gammas = [1, 2, 10, 20]
mspe_array = np.zeros((len(alphas), len(gammas)))

# Iterate over alphas and gammas, and fill the MSPE values
for alpha in alphas:
    for gamma in gammas:

        model = KernelRidge(alpha=alpha, kernel="polynomial", gamma = gamma, coef0=1)

        crossValErrors = kFoldMspeCalc(kf, model, X, y)
        # Print mean MSPE
        print("Polynomial Kernel MSPE with lambda: "+str(alpha)+" and degree: "+str(gamma)+" is:" ,np.mean(crossValErrors))
    print()

print("lambda =1 yields the best results. Running predictions with lambda =1")
for gamma in gammas:
    model = KernelRidge(alpha=1, kernel="polynomial", gamma = gamma, coef0=1)
    print("Fitting on Testing Data and degreee: "+str(gamma)+": ", testingMSPE(model, X_test, y_test))


Polynomial Kernel MSPE with lambda: 0 and degree: 1 is: 392.5130814353853
Polynomial Kernel MSPE with lambda: 0 and degree: 2 is: 3127.9680979666437
Polynomial Kernel MSPE with lambda: 0 and degree: 10 is: 563.8374225738235
Polynomial Kernel MSPE with lambda: 0 and degree: 20 is: 298.876355522254

Polynomial Kernel MSPE with lambda: 0.1 and degree: 1 is: 0.9478480473229165
Polynomial Kernel MSPE with lambda: 0.1 and degree: 2 is: 0.945033533703986
Polynomial Kernel MSPE with lambda: 0.1 and degree: 10 is: 1.005677640441731
Polynomial Kernel MSPE with lambda: 0.1 and degree: 20 is: 1.0678771844511354

Polynomial Kernel MSPE with lambda: 0.5 and degree: 1 is: 0.9541942347464376
Polynomial Kernel MSPE with lambda: 0.5 and degree: 2 is: 0.9467320045850471
Polynomial Kernel MSPE with lambda: 0.5 and degree: 10 is: 0.9897347163833563
Polynomial Kernel MSPE with lambda: 0.5 and degree: 20 is: 1.0181877000587871

Polynomial Kernel MSPE with lambda: 1 and degree: 1 is: 0.9546783560102865
Polyno

Polynomial features with degree 2 seem to give the best testing performance and it seems that the data set is prone to overfitting


Both methods aim to capture non-linear relationships in the data by introducing higher-order terms or mapping the data to a higher-dimensional feature space. 

In polynomial regression, we explicitly create the higher-order terms, which can become computationally expensive and lead to the curse of dimensionality as the degree of the polynomial increases. 

Kernel ridge regression with a polynomial kernel uses the kernel trick to implicitly map the data to a higher-dimensional space, which can be more computationally efficient and less prone to the curse of dimensionality.

With certain regularization, the kernel performs slightly better, although higher degrees do not seem to lead to higher performance



<h3>4. (10 pts) How does the performance change if we use the radial basis function and Laplace (or exponential) kernels in the previous question?</h3>


In [34]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.kernel_ridge import KernelRidge

# Ridge penalty values
alphas = [0, 0.1, 0.5, 1]

# Polynomial degree values
gammas = [1, 2, 10, 20]
mspe_array = np.zeros((len(alphas), len(gammas)))

# Iterate over alphas and gammas, and fill the MSPE values
for alpha in alphas:
    for gamma in gammas:

        model = KernelRidge(alpha=alpha, kernel="rbf", gamma = gamma)

        crossValErrors = kFoldMspeCalc(kf, model, X, y)
        # Print mean MSPE
        print("Polynomial Kernel MSPE with lambda: "+str(alpha)+" and degree: "+str(gamma)+" is:" ,np.mean(crossValErrors))
    print()

print("lambda =1 yields the best results. Running predictions with lambda =0.1")
for gamma in gammas:
    model = KernelRidge(alpha=0.1, kernel="rbf", gamma = gamma, coef0=1)
    print("Fitting on Testing Data and degreee: "+str(gamma)+": ", testingMSPE(model, X_test, y_test))


Polynomial Kernel MSPE with lambda: 0 and degree: 1 is: 279191.5477681823
Polynomial Kernel MSPE with lambda: 0 and degree: 2 is: 6071685.852212243
Polynomial Kernel MSPE with lambda: 0 and degree: 10 is: 68787579.24764112
Polynomial Kernel MSPE with lambda: 0 and degree: 20 is: 802963844.3469262

Polynomial Kernel MSPE with lambda: 0.1 and degree: 1 is: 0.9563927390319306
Polynomial Kernel MSPE with lambda: 0.1 and degree: 2 is: 0.9516024295480948
Polynomial Kernel MSPE with lambda: 0.1 and degree: 10 is: 0.9941336414469303
Polynomial Kernel MSPE with lambda: 0.1 and degree: 20 is: 1.0514991774381808

Polynomial Kernel MSPE with lambda: 0.5 and degree: 1 is: 0.9747783163358681
Polynomial Kernel MSPE with lambda: 0.5 and degree: 2 is: 0.9693558186727567
Polynomial Kernel MSPE with lambda: 0.5 and degree: 10 is: 1.0041811775267389
Polynomial Kernel MSPE with lambda: 0.5 and degree: 20 is: 1.076659020290951

Polynomial Kernel MSPE with lambda: 1 and degree: 1 is: 0.9914488735424213
Polyn

In [35]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.kernel_ridge import KernelRidge

# Ridge penalty values
alphas = [0, 0.1, 0.5, 1]

# Polynomial degree values
gammas = [1, 2, 10, 20]
mspe_array = np.zeros((len(alphas), len(gammas)))

# Iterate over alphas and gammas, and fill the MSPE values
for alpha in alphas:
    for gamma in gammas:

        model = KernelRidge(alpha=alpha, kernel="laplacian", gamma = gamma, coef0=1)

        crossValErrors = kFoldMspeCalc(kf, model, X, y)
        # Print mean MSPE
        print("Polynomial Kernel MSPE with lambda: "+str(alpha)+" and degree: "+str(gamma)+" is:" ,np.mean(crossValErrors))
    print()

print("lambda =1 yields the best results. Running predictions with lambda =0.5")
for gamma in gammas:
    model = KernelRidge(alpha=0.5, kernel="laplacian", gamma = gamma)
    print("Fitting on Testing Data and degreee: "+str(gamma)+": ", testingMSPE(model, X_test, y_test))


Polynomial Kernel MSPE with lambda: 0 and degree: 1 is: 1.9243062863214973
Polynomial Kernel MSPE with lambda: 0 and degree: 2 is: 1.801786290010964
Polynomial Kernel MSPE with lambda: 0 and degree: 10 is: 1.9005569190553142
Polynomial Kernel MSPE with lambda: 0 and degree: 20 is: 2.412808307906997

Polynomial Kernel MSPE with lambda: 0.1 and degree: 1 is: 1.042216373223147
Polynomial Kernel MSPE with lambda: 0.1 and degree: 2 is: 1.1137096215980862
Polynomial Kernel MSPE with lambda: 0.1 and degree: 10 is: 1.7285373407054454
Polynomial Kernel MSPE with lambda: 0.1 and degree: 20 is: 2.375799882746372

Polynomial Kernel MSPE with lambda: 0.5 and degree: 1 is: 0.9960367493127593
Polynomial Kernel MSPE with lambda: 0.5 and degree: 2 is: 1.0427232175527847
Polynomial Kernel MSPE with lambda: 0.5 and degree: 10 is: 1.6942291353516439
Polynomial Kernel MSPE with lambda: 0.5 and degree: 20 is: 2.425690652141559

Polynomial Kernel MSPE with lambda: 1 and degree: 1 is: 1.0026442320335334
Polyn

RBF does not significantly outperform the polynomial kernel at degree 2, it would appear polynomial kernels do appear to get significant performance

<h3>5. (10 pts) Investigate the utility of random Fourier feature expansion in improving the prediction
of motor UPDRS scores for the polynomial feature maps of order 2 and 5, respectively. Vary the
random feature dimension as 6, 12, and 24. How do the random features compare to previous
methods?</h3>

In [36]:
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

# Create a list of random feature dimensions
random_feature_dims = [6, 12, 24]


# Loop over polynomial degrees
for degree in [2, 5]:
        
        # Create a PolynomialFeatures object
        poly = PolynomialFeatures(degree=degree)
        
        # Loop over random feature dimensions
        for n_components in random_feature_dims:       
            # Create a pipeline with RBFSampler and Ridge regression
            rbf_sampler = RBFSampler(n_components=n_components, gamma=1.0)
            ridge = Ridge()
            model = Pipeline([("rbf_sampler", rbf_sampler),
                            ("poly_features", poly),
                            ("ridge", ridge)])
            
            # Perform 10-fold cross-validation and compute the MSPE
            crossValErrors = kFoldMspeCalc(kf, model, X, y)
            mspe = np.mean(crossValErrors)
            print("RBF Kernel with "+str(n_components)+" random features with polynomial degree "+str(degree)+" is:" ,np.mean(crossValErrors))
print()

degree = 5
poly = PolynomialFeatures(degree=degree)
rbf_sampler = RBFSampler(n_components=n_components, gamma=1.0)
model = Pipeline([("rbf_sampler", rbf_sampler),
                            ("poly_features", poly),
                            ("ridge", ridge)])

print("Fitting on Testing Data: ", testingMSPE(model, X_test, y_test))

RBF Kernel with 6 random features with polynomial degree 2 is: 1.0330527877498334


RBF Kernel with 12 random features with polynomial degree 2 is: 0.981770286108666
RBF Kernel with 24 random features with polynomial degree 2 is: 0.970854962484076
RBF Kernel with 6 random features with polynomial degree 5 is: 0.9844314154178073
RBF Kernel with 12 random features with polynomial degree 5 is: 0.9890882083542618
RBF Kernel with 24 random features with polynomial degree 5 is: 0.96286860633002

Fitting on Testing Data:  1.1216076055097337


Random Fourier features do give a pretty good performance on the data set

<h3>6. (10 pts) Fit a two-layered (shallow) neural network for predicting motor UPDRS scores. Is its performance better than that of the previous methods? Justify the choice of tuning parameters such as number of hidden units, pre-activation function, learning rate, and epochs.</h3>

In [44]:
from sklearn.neural_network import MLPRegressor

for size in [16]:
    for activation in ["identity", "relu", "tanh", "logistic"]:
        for learning_rate in [0.001, 0.01, 0.1]:
            # Create the neural network model
            model = MLPRegressor(
                hidden_layer_sizes=(size,),  # Number of hidden units
                activation=activation,  # Activation function for hidden layers
                solver='adam',  # Optimization algorithm
                alpha=0.0001,  # L2 regularization parameter
                batch_size='auto',  # Batch size for optimization
                learning_rate_init=learning_rate,  # Initial learning rate
                max_iter=500,  # Maximum number of iterations
                shuffle=True,  # Shuffle the data before each iteration
                random_state=42,  # Random state for reproducibility
                n_iter_no_change=10  # Maximum number of iterations with no improvement
            )

            crossValErrors = kFoldMspeCalc(kf, model, X, y)

            # Print mean MSPE
            print("2 Layer NN with "+str(size)+", activation "+activation+" learning_rate:"+str(learning_rate)+" MSPE is:" ,np.mean(crossValErrors))


2 Layer NN with 16, activation identity learning_rate:0.001 MSPE is: 1.378090734400517
2 Layer NN with 16, activation identity learning_rate:0.01 MSPE is: 0.9938879032116505
2 Layer NN with 16, activation identity learning_rate:0.1 MSPE is: 1.1823460416531895
2 Layer NN with 16, activation relu learning_rate:0.001 MSPE is: 1.262759670845105
2 Layer NN with 16, activation relu learning_rate:0.01 MSPE is: 1.0492827125372273
2 Layer NN with 16, activation relu learning_rate:0.1 MSPE is: 1.3495954958700165
2 Layer NN with 16, activation tanh learning_rate:0.001 MSPE is: 1.405237626351632
2 Layer NN with 16, activation tanh learning_rate:0.01 MSPE is: 1.0288073409279996
2 Layer NN with 16, activation tanh learning_rate:0.1 MSPE is: 1.166715972571512
2 Layer NN with 16, activation logistic learning_rate:0.001 MSPE is: 1.8154281419371994
2 Layer NN with 16, activation logistic learning_rate:0.01 MSPE is: 1.013769475687628
2 Layer NN with 16, activation logistic learning_rate:0.1 MSPE is: 1.18

Activation function identity(linear) and learning rate: 0.01 is the best

In [48]:
 
for size in [16, 64, 256, 1024, 4096]:
    for activation in ["identity"]:
        for learning_rate in [0.01]:
            # Create the neural network model
            model = MLPRegressor(
                hidden_layer_sizes=(size,),  # Number of hidden units
                activation=activation,  # Activation function for hidden layers
                solver='adam',  # Optimization algorithm
                alpha=0.0001,  # L2 regularization parameter
                batch_size='auto',  # Batch size for optimization
                learning_rate_init=learning_rate,  # Initial learning rate
                max_iter=500,  # Maximum number of iterations
                shuffle=True,  # Shuffle the data before each iteration
                random_state=42,  # Random state for reproducibility
                n_iter_no_change=10  # Maximum number of iterations with no improvement
            )

            crossValErrors = kFoldMspeCalc(kf, model, X, y)

            # Print mean MSPE
            print("2 Layer NN with "+str(size)+", activation "+activation+" learning_rate:"+str(learning_rate)+" MSPE is:" ,np.mean(crossValErrors))




model = MLPRegressor(
                hidden_layer_sizes=(16,),  # Number of hidden units
                activation="identity",  # Activation function for hidden layers
                solver='adam',  # Optimization algorithm
                alpha=0.0001,  # L2 regularization parameter
                batch_size='auto',  # Batch size for optimization
                learning_rate_init=0.01,  # Initial learning rate
                max_iter=500,  # Maximum number of iterations
                shuffle=True,  # Shuffle the data before each iteration
                random_state=42,  # Random state for reproducibility
                n_iter_no_change=10  # Maximum number of iterations with no improvement
            )
print("Fitting on Testing Data with identity @ 0.01 learning rate and 16 hidden units: ", testingMSPE(model, X_test, y_test))

2 Layer NN with 16, activation identity learning_rate:0.01 MSPE is: 0.9938879032116505
2 Layer NN with 64, activation identity learning_rate:0.01 MSPE is: 1.0540220970488563
2 Layer NN with 256, activation identity learning_rate:0.01 MSPE is: 1.0257832423290403
2 Layer NN with 1024, activation identity learning_rate:0.01 MSPE is: 1.0427417934360284
2 Layer NN with 4096, activation identity learning_rate:0.01 MSPE is: 1.4157505246558837
Fitting on Testing Data with identity @ 0.01 learning rate and 64 hidden units:  1.160988882407963


0.01 initial learning rate with relu activation with 16 hidden units is best in terms of training and testing

Number of hidden units: The number of hidden units (64 in this example) controls the complexity of the neural network model. More hidden units can capture more complex patterns in the data.

Activation function: The Identity activation function is a commonly used in regression tasks.

Learning rate: The learning rate (0.001 in this example) controls the step size of the optimization algorithm. A smaller learning rate may converge more slowly but may be more stable, while a larger learning rate may converge faster but may be less stable. You can try different values or use a learning rate scheduler.

Epochs (max_iter): The number of epochs (500 in this example) determines how many times the entire dataset is used for training. More epochs may lead to better performance but also increase the risk of overfitting.

<h3>7. (10 pts) Compare the predictive performance when a deep neural network replaces the shallow
neural networks.</h3>

In [49]:
# Create the neural network model
model = MLPRegressor(
    hidden_layer_sizes=(8,8,8,8,8),  # 8 hidden units in 5 layers
    activation='relu',  # Activation function for hidden layers
    solver='adam',  # Optimization algorithm
    alpha=0.0001,  # L2 regularization parameter
    batch_size='auto',  # Batch size for optimization
    learning_rate_init=0.01,  # Initial learning rate
    max_iter=500,  # Maximum number of iterations
    shuffle=True,  # Shuffle the data before each iteration
    random_state=42,  # Random state for reproducibility
    n_iter_no_change=10  # Maximum number of iterations with no improvement
)

crossValErrors = kFoldMspeCalc(kf, model, X, y)

# Print mean MSPE
print("5 Layer NN MSPE is:" ,np.mean(crossValErrors))

print("Fitting on 5 layer netwrok with relu @ 0.01 learning rate : ", testingMSPE(model, X_test, y_test))

5 Layer NN MSPE is: 1.0525286133780953
Fitting on 5 layer netwrok with relu @ 0.01 learning rate :  1.125447688230306


In [50]:
model = MLPRegressor(
    hidden_layer_sizes=(16,16,16,16,16),  # 16 hidden units in 5 layers
    activation='relu',  # Activation function for hidden layers
    solver='adam',  # Optimization algorithm
    alpha=0.0001,  # L2 regularization parameter
    batch_size='auto',  # Batch size for optimization
    learning_rate_init=0.01,  # Initial learning rate
    max_iter=500,  # Maximum number of iterations
    shuffle=True,  # Shuffle the data before each iteration
    random_state=42,  # Random state for reproducibility
    n_iter_no_change=10  # Maximum number of iterations with no improvement
)

crossValErrors = kFoldMspeCalc(kf, model, X, y)

# Print mean MSPE
print("5 Layer NN MSPE is:" ,np.mean(crossValErrors))

print("Fitting on 5 layer netwrok with relu @ 0.01 learning rate : ", testingMSPE(model, X_test, y_test))

5 Layer NN MSPE is: 1.0463081168309047
Fitting on 5 layer netwrok with relu @ 0.01 learning rate :  1.1208587261879228


In [51]:
model = MLPRegressor(
    hidden_layer_sizes=(16,16,16,16,16, 16, 16,16,16,16),  # 16 hidden units in 10 layers
    activation='relu',  # Activation function for hidden layers
    solver='adam',  # Optimization algorithm
    alpha=0.0001,  # L2 regularization parameter
    batch_size='auto',  # Batch size for optimization
    learning_rate_init=0.01,  # Initial learning rate
    max_iter=500,  # Maximum number of iterations
    shuffle=True,  # Shuffle the data before each iteration
    random_state=42,  # Random state for reproducibility
    n_iter_no_change=10  # Maximum number of iterations with no improvement
)

crossValErrors = kFoldMspeCalc(kf, model, X, y)

# Print mean MSPE
print("5 Layer NN MSPE is:" ,np.mean(crossValErrors))

print("Fitting on 5 layer netwrok with relu @ 0.01 learning rate : ", testingMSPE(model, X_test, y_test))

5 Layer NN MSPE is: 1.3712590882154783
Fitting on 5 layer netwrok with relu @ 0.01 learning rate :  1.2790406570357609


5 layers with 16 hidden units seems to work best

<h3>8. (10 pts) Assess and compare the MSPEs of all the regression models from (i) to (vii). Comment
on this flexibility and their dependence on the tuning parameters. For example, does the ranks
of MSPEs match the rankings of the flexibility?</h3>

Flexibility does seem to effect testing MSPE generally as we went from having a 1.18 testing MSPE on linear values to an MSPE of approx 1.12 with neural networks. Some regressors were highly dependent on tuning params like RBF but overall the performance was not drastically different betweeen models

<h3>9. (10 pts) Perform a sensitivity analysis on the tuning parameters, such as degree of the polynomial, regularization parameter, random feature dimension, learning rate, number of layers, and number of epochs. How do changes in these parameters affect the model’s performance? Which model balances sensitivity to the choice of tuning parameter and predictive accuracy?</h3>

Kernels with penalties like poynomial ridge and PCA were dependent on their tuning. Increasing model complexity i.e. model degree does not always help as bais variance tradeoff seems to be at play